In [12]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
from crewai import Agent, Task, Crew
from dotenv import load_dotenv
from crewai import LLM
from langchain_community.llms import HuggingFaceHub
import os

load_dotenv()

llm = LLM(
    model="mistral/mistral-small-latest",  # ✅ Chat model
    api_key=os.getenv("MISTRAL_API_KEY")
)



## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

#### Planner Agent

In [14]:
planner = Agent(
    role = "Content Planner",
    goal = "Plan a content piece based on the provided {topic}.",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",   
    allow_delegation = False,
    verbose = True,
    llm = llm
)

#### Writer Agent


In [15]:
writer = Agent(
    role = "Content Writer",
    goal = "Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm = llm
)

#### Editor Agent

In [16]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm = llm
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

#### Plan Task

In [17]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

#### Write Task

In [18]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

#### Edit Task

In [19]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [20]:
crew = Crew(
    agents = [planner, writer, editor],
    tasks = [plan, write, edit],
    verbose = True,
)

## Runnning the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [21]:
result = crew.kickoff(inputs = {"topic": "The Future of AI in Everyday Life"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 26928389-9870-4ec6-82d5-8571804dad43                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Task: 1. Prioritize the latest trends, key players, and noteworthy news on The Future of AI in Everyday Life.  │
│  2. Identify the target audience, considering their interests and pain points.                                  │
│  3. Develop a detailed content outline including an introduction, key points, and a call to action.             │
│  4. Include SEO keywords and relevant data or sources.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Content Plan: The Future of AI in Everyday Life**                                                            │
│                                                                                                                 │
│  **1. Latest Trends, Key Players, and Noteworthy News**                                                         │
│                                                                                                                 │
│  **Trends:**                                                                                                    │
│  - AI in Smart Homes: Integration of AI in home automation, security, and energy management.                    │
│  - AI in Healthcare: Personalized medicine, AI-assisted diagnostics, and robotic surgery.                       │
│  - AI in Education: Personalized learning, AI tutors, and administrative task automation.                       │
│  - AI in Transportation: Autonomous vehicles, route optimization, and predictive maintenance.                   │
│  - AI in Entertainment: Personalized content recommendations, AI-generated art, and virtual assistants.         │
│  - Ethical AI: Increased focus on AI ethics, bias, and transparency.                                            │
│                                                                                                                 │
│  **Key Players:**                                                                                               │
│  - Tech Giants: Google, Apple, Microsoft, Amazon, IBM.                                                          │
│  - Startups: Specialized AI startups like DeepMind, OpenAI, and numerous others focusing on niche               │
│  applications.                                                                                                  │
│  - Research Institutions: MIT, Stanford, and other universities leading AI research.                            │
│                                                                                                                 │
│  **Noteworthy News:**                                                                                           │
│  - Advancements in AI hardware, such as more powerful and efficient AI chips.                                   │
│  - Increased investment in AI startups and research.                                                            │
│  - Growing concerns about AI ethics and regulation.                                                             │
│                                                                                                                 │
│  **2. Target Audience Analysis**                                                                                │
│                                                                                                                 │
│  **Primary Audience:**                                                                                          │
│  - Tech Enthusiasts: Individuals interested in the latest AI trends and applications.                           │
│  - Professionals: Those working in fields where AI is becoming increasingly relevant (healthcare, education,    │
│  transportation, etc.).                                                                                         │
│  - Consumers: People who use AI-powered devices and ser

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: d6211e3f-0d6d-4891-8a1f-ea5c54a38dcd                                                                     │
│  Agent: Content Planner                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Here is the edited blog post in markdown format, ready for publication:                                        │
│                                                                                                                 │
│  ```markdown                                                                                                    │
│  # The Future of AI in Everyday Life                                                                            │
│                                                                                                                 │
│  Artificial Intelligence (AI) has seamlessly integrated into our daily lives, transforming the way we live,     │
│  work, and interact with the world. From smart home devices to personalized healthcare, AI's influence is       │
│  undeniable. As we look ahead, it's crucial to understand the future trajectory of AI and its potential impact  │
│  on our everyday experiences.                                                                                   │
│                                                                                                                 │
│  ## AI in Smart Homes: The Future of Convenience and Security                                                   │
│                                                                                                                 │
│  AI-powered home devices have become commonplace, with smart speakers, thermostats, and security systems        │
│  enhancing our daily routines. These devices offer unparalleled convenience, energy efficiency, and security.   │
│  Imagine a home where lights adjust to your preferences, thermostats optimize energy usage based on your        │
│  habits, and security systems proactively detect and deter threats. This is not a distant dream but a           │
│  near-future reality.                                                                                           │
│                                                                                                                 │
│  Looking ahead, we can expect even more integrated and autonomous home systems. AI will learn and adapt to our  │
│  behaviors, creating a seamless and personalized living environment. For instance, AI could anticipate your     │
│  needs, such as brewing your morning coffee or adjusting the room temperature before you wake up. The future    │
│  of smart homes lies in their ability to anticipate and respond to our needs, making our lives more             │
│  comfortable and secure.                                                                                        │
│                                                                                                                 │
│  ## AI in Healthcare: Revolutionizing Patient Care                                                              │
│                                                                                                                 │
│  The healthcare industry is undergoing a significant transformation with the integration of AI. AI-assisted     │
│  diagnostics, personalized medicine, and robotic surgery are just a few examples of how AI is improving         │
│  patient outcomes. AI algorithms can analyze vast amounts of medical data to provide accurate diagnoses and     │
│  treatment plans, often faster than human doctors.     

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: d78741d9-8361-4d2c-b358-a9e2b707c1f5                                                                     │
│  Agent: Editor                                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 26928389-9870-4ec6-82d5-8571804dad43                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Here is the edited blog post in markdown format, ready for publication:                          │
│                                                                                                                 │
│  ```markdown                                                                                                    │
│  # The Future of AI in Everyday Life                                                                            │
│                                                                                                                 │
│  Artificial Intelligence (AI) has seamlessly integrated into our daily lives, transforming the way we live,     │
│  work, and interact with the world. From smart home devices to personalized healthcare, AI's influence is       │
│  undeniable. As we look ahead, it's crucial to understand the future trajectory of AI and its potential impact  │
│  on our everyday experiences.                                                                                   │
│                                                                                                                 │
│  ## AI in Smart Homes: The Future of Convenience and Security                                                   │
│                                                                                                                 │
│  AI-powered home devices have become commonplace, with smart speakers, thermostats, and security systems        │
│  enhancing our daily routines. These devices offer unparalleled convenience, energy efficiency, and security.   │
│  Imagine a home where lights adjust to your preferences, thermostats optimize energy usage based on your        │
│  habits, and security systems proactively detect and deter threats. This is not a distant dream but a           │
│  near-future reality.                                                                                           │
│                                                                                                                 │
│  Looking ahead, we can expect even more integrated and autonomous home systems. AI will learn and adapt to our  │
│  behaviors, creating a seamless and personalized living environment. For instance, AI could anticipate your     │
│  needs, such as brewing your morning coffee or adjusting the room temperature before you wake up. The future    │
│  of smart homes lies in their ability to anticipate and respond to our needs, making our lives more             │
│  comfortable and secure.                                                                                        │
│                                                                                                                 │
│  ## AI in Healthcare: Revolutionizing Patient Care                                                              │
│                                                                                                                 │
│  The healthcare industry is undergoing a significant transformation with the integration of AI. AI-assisted     │
│  diagnostics, personalized medicine, and robotic surgery are just a few examples of how AI is improving         │
│  patient outcomes. AI algorithms can analyze vast amou

## Key Points to Remember
- Agents perform better when role-playing.
- You should focus on goals and expectations.
- One agent can do multiple tasks.
- Tasks and agents should be granular.
- Tasks can be executed in different ways.

In [24]:
from IPython.display import Markdown
Markdown(result.raw)

Here is the edited blog post in markdown format, ready for publication:

```markdown
# The Future of AI in Everyday Life

Artificial Intelligence (AI) has seamlessly integrated into our daily lives, transforming the way we live, work, and interact with the world. From smart home devices to personalized healthcare, AI's influence is undeniable. As we look ahead, it's crucial to understand the future trajectory of AI and its potential impact on our everyday experiences.

## AI in Smart Homes: The Future of Convenience and Security

AI-powered home devices have become commonplace, with smart speakers, thermostats, and security systems enhancing our daily routines. These devices offer unparalleled convenience, energy efficiency, and security. Imagine a home where lights adjust to your preferences, thermostats optimize energy usage based on your habits, and security systems proactively detect and deter threats. This is not a distant dream but a near-future reality.

Looking ahead, we can expect even more integrated and autonomous home systems. AI will learn and adapt to our behaviors, creating a seamless and personalized living environment. For instance, AI could anticipate your needs, such as brewing your morning coffee or adjusting the room temperature before you wake up. The future of smart homes lies in their ability to anticipate and respond to our needs, making our lives more comfortable and secure.

## AI in Healthcare: Revolutionizing Patient Care

The healthcare industry is undergoing a significant transformation with the integration of AI. AI-assisted diagnostics, personalized medicine, and robotic surgery are just a few examples of how AI is improving patient outcomes. AI algorithms can analyze vast amounts of medical data to provide accurate diagnoses and treatment plans, often faster than human doctors.

In the future, AI-driven healthcare systems will become more prevalent, offering personalized treatment plans tailored to individual patients. Imagine a world where AI can predict and prevent illnesses before they occur, or where robotic surgeons perform complex operations with precision and accuracy. The potential for AI in healthcare is immense, promising a future where healthcare is more accessible, efficient, and effective.

## AI in Education: Personalized Learning for All

Education is another sector where AI is making a significant impact. Personalized learning platforms, AI tutors, and administrative task automation are enhancing the educational experience for students and educators alike. AI can adapt to a student's learning pace and style, providing customized lessons and feedback to improve learning outcomes.

As we look to the future, AI-driven educational platforms will become more sophisticated, offering immersive and interactive learning experiences. Virtual classrooms will enable students from around the world to learn together, breaking down geographical barriers. AI tutors will become more advanced, providing real-time feedback and support to students. The future of education lies in its ability to adapt to individual needs, making learning more engaging and effective.

## AI in Transportation: Paving the Way for Smart Cities

The transportation sector is also undergoing a transformation with the advent of AI. Autonomous vehicles, route optimization, and predictive maintenance are just a few examples of how AI is improving transportation efficiency and safety. AI-powered navigation systems can optimize routes in real-time, reducing traffic congestion and travel time.

In the future, fully autonomous transportation systems will become a reality, paving the way for smart cities. Imagine a world where self-driving cars, buses, and trains navigate our streets seamlessly, reducing accidents and improving traffic flow. AI will also play a crucial role in predictive maintenance, ensuring that vehicles and infrastructure are well-maintained and safe. The future of transportation lies in its ability to adapt to our needs, making travel more efficient and sustainable.

## AI in Entertainment: Enhancing User Experiences

The entertainment industry is not left behind in the AI revolution. Personalized content recommendations, AI-generated art, and virtual assistants are enhancing user experiences and increasing engagement. AI algorithms analyze user preferences to recommend movies, music, and other content tailored to individual tastes.

Looking ahead, AI-driven entertainment platforms will offer immersive and interactive experiences. Imagine a world where AI-generated art and music create new forms of entertainment, or where virtual assistants provide personalized recommendations and support. The future of entertainment lies in its ability to adapt to our preferences, making our leisure time more enjoyable and engaging.

## Ethical AI: Navigating the Challenges Ahead

As AI becomes more integrated into our lives, ethical considerations become increasingly important. Concerns about bias, transparency, and privacy issues are at the forefront of the AI debate. It's crucial to address these challenges to ensure that AI is developed and used responsibly.

In the future, ethical guidelines, regulations, and AI governance frameworks will play a vital role in shaping the development and deployment of AI. Imagine a world where AI is fair, transparent, and accountable, earning the trust of users and stakeholders. The future of AI lies in its ability to address ethical concerns, ensuring that it is used for the benefit of all.

## Conclusion: Embracing the Future of AI

The future of AI in everyday life is bright and full of possibilities. From smart homes to healthcare, education, transportation, and entertainment, AI is set to transform our daily experiences. As we embrace these changes, it's essential to stay informed and engaged with AI developments. By understanding the potential of AI and addressing its challenges, we can shape a future where AI enhances our lives in meaningful and positive ways.

## Call to Action

We encourage you to share your thoughts and experiences with AI. How has AI impacted your daily life? What do you envision for the future of AI? Join the conversation and share your insights in the comments below.

To stay updated on the latest AI trends and developments, sign up for our newsletter or follow our blog. We provide valuable resources and insights to help you navigate the ever-evolving world of AI.

For further exploration, check out these additional resources:

- [AI in Everyday Life: A Comprehensive Guide](https://example.com/ai-in-everyday-life)
- [The Future of AI: Trends and Predictions](https://example.com/future-of-ai)
- [Ethical AI: Navigating the Challenges](https://example.com/ethical-ai)

Join us in embracing the future of AI and shaping a world where technology enhances our lives in meaningful and positive ways.
```